# 搭建yolo v3
注：该文件仅记录搭建网络的步骤，全部用markdown写成
## 配置文件
配置文件保存了yolo网络的全部网络参数，保存于/cfg/yolov3.cfg中

在cgf文件中可以看到，YOLO共用了五种层结构：
### Convolutional
```cfg
[convolutional]
batch_normalize=1 
filters=64 
size=3 
stride=1 
pad=1 
activation=leaky
```
除上面代码提到的外，还有另外两个参数不同的卷积层结构。
### Shortcut
```cfg
[shortcut]
from=-3 
activation=linear 
```
shortcut层是一个跳过连接操作，在残差神经网络ResNet中有出现，具体可见吴恩达的深度学习课程，我后面也会写一篇讲ResNet的文章（又在给自己挖坑了😑）
### Upsample
```cfg
[upsample]
stride=2
```
用以双线性上采样
### Route
```cfg
[route]
layers = -4

[route]
layers = -1, 61
```
路由层，它的参数 layers 有一个或两个值。
- 仅有一个值时，输出这一层通过该值索引的特征图，这里设置为-4，所以层级将输出路由层之前第四个层的特征图。
- 当层级有两个值时，它将返回由这两个值索引的拼接特征图，这里设置为为-1 和 61，因此该层级将输出从前一层级（-1）到第 61 层的特征图，并将它们按深度拼接。

### YOLO
```cfg
[yolo]
mask = 0,1,2
anchors = 10,13, 16,30, 33,23, 30,61, 62,45, 59,119, 116,90, 156,198, 373,326
classes=80
num=9
jitter=.3
ignore_thresh = .5
truth_thresh = 1
random=1
```
yolo层级用于最后的目标检测。参数 anchors 定义了 9 组锚点，但是它们只是由 mask 标签使用的属性所索引的锚点。这里，mask 的值为 0、1、2 表示了第一个、第二个和第三个使用的锚点，表示每一个单元格预测三个边框。总而言之，预测层的规模为3，共9个锚点。
### Net
还有一个部分net，只描述网络输入和训练参数的相关信息，并未用于 YOLO 的前向传播，所以并不计算为层。但这一部分它为我们提供了网络输入大小等信息，可用于调整前向传播中的锚点。
```cfg
[net]
# Testing
batch=1
subdivisions=1
# Training
# batch=64
# subdivisions=16
width= 320
height = 320
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1
```

## 解析配置文件
需要创建一个darknet.py文件，Darknet是构建 YOLO 底层架构的环境（这个架构同样也由该作者开发），这个文件将包含实现 YOLO 网络的所有代码。同样我们还需要补充一个名为 util.py 的文件，它会包含多种需要调用的函数。在将所有这些文件保存在检测器文件夹下后，我们就能使用 git 追踪它们的改变。

在darknet.py 文件顶部添加必要的导入项。
```python
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
```
我们定义一个函数 parse_cfg，该函数使用配置文件的路径作为输入。
```python
def parse_cfg(cfgfile):
```
该函数用来解析cfg，将每个块存储为词典
```python
file = open(cfgfile, 'r')
lines = file.read().split('\n')         # store the lines in a list
lines = [x for x in lines if len(x) > 0]   # get read of the empty lines 
lines = [x for x in lines if x[0] != '#']  # get rid of comments
lines = [x.rstrip().lstrip() for x in lines]# get rid of fringe whitespaces
```